# Titanic

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB

In [63]:
test = pd.read_csv("Data/test.csv")
train = pd.read_csv("Data/train.csv")

In [64]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Priprava podatkov

In [65]:
train = train[np.invert(np.isnan(train["Age"]))]
test["Age"] = test["Age"].fillna(np.mean(test["Age"]))
test["Fare"] = test["Fare"].fillna(np.mean(test["Fare"]))
#test = test[np.invert(np.isnan(test["Age"]))]
#test = test[np.invert(np.isnan(test["Fare"]))]

Nov atribut minor:

In [66]:
def minors(x):
    minor = []
    for a in x["Age"]:
        if a < 16:
            minor = np.append(minor, 1)
        else:
            minor = np.append(minor, 0)
    x["Minor"] = minor

In [67]:
minors(train)
minors(test)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Nova atributa na podlagi priimkov:
* Vsaj en prezivel z istim priimkom
* Vsi umrli z istim priimkom

In [68]:
def allDied(x, y, ista):
    allD = []
    lived = []
    for index, row in x.iterrows():
        surname = row["Name"].split(",")[0]
        dead = 0
        alive = 0
        for index2, row2 in y.iterrows():
            if ista:
                if index != index2:
                    surname2 = row2["Name"].split(",")[0]
                    if surname2 == surname:
                        if row2["Survived"] == 1:
                            alive += 1
                        else:
                            dead += 1
            else:
                surname2 = row2["Name"].split(",")[0]
                if surname2 == surname:
                    if row2["Survived"] == 1:
                        alive += 1
                    else:
                        dead += 1
        if dead + alive < 2:
            allD = np.append(allD, 0)
            lived = np.append(lived, 0)
        else:
            if alive == 0:
                allD = np.append(allD, 1)
            else:
                allD = np.append(allD, 0)
            if alive >= 1:
                lived = np.append(lived, 1)
            else:
                lived = np.append(lived, 0)
    x["SurnameAllDied"] = allD
    x["SurnameOneLived"] = lived

In [69]:
allDied(train, train, True)
allDied(test, train, False)

Tabele za napovedovanje:

In [70]:
def one_hot(name, x, y):
    category = np.array([])
    for a in y[name]:
        if a not in category:
            category = np.append(category, a)
    category = np.sort(category)
    for c in category:
        current = []
        for a in y[name]:
            if a == c:
                current = np.append(current, 1)
            else:
                current = np.append(current, 0)
        x[c] = current

In [71]:
y = train["Survived"]
x = train[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"]]
x["FamilySize"] = train["Parch"] + train["SibSp"]
one_hot("Sex", x, train)
#one_hot("Embarked", x, train)
x = x.loc[:, x.columns != 'nan']
#enc = OneHotEncoder()
#enc.fit(train)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/tmp/ipykernel_5983/1030216102.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [72]:
xtest = test[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"]]
xtest["FamilySize"] = test["Parch"] + test["SibSp"]
one_hot("Sex", xtest, test)
#one_hot("Embarked", xtest, test)

/tmp/ipykernel_5983/1030216102.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [73]:
x

,Pclass,Age,Fare,Minor,SurnameAllDied,SurnameOneLived,SibSp,Parch,FamilySize,female,male
0,3,22.0,7.2500,0.0,0.0,0.0,1,0,1,0.0,1.0
1,1,38.0,71.2833,0.0,0.0,0.0,1,0,1,1.0,0.0
2,3,26.0,7.9250,0.0,0.0,0.0,0,0,0,1.0,0.0
3,1,35.0,53.1000,0.0,0.0,0.0,1,0,1,1.0,0.0
4,3,35.0,8.0500,0.0,0.0,0.0,0,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
885,3,39.0,29.1250,0.0,1.0,0.0,0,5,5,1.0,0.0
886,2,27.0,13.0000,0.0,0.0,0.0,0,0,0,0.0,1.0
887,1,19.0,30.0000,0.0,0.0,1.0,0,0,0,1.0,0.0
889,1,26.0,30.0000,0.0,0.0,0.0,0,0,0,0.0,1.0


In [74]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Minor,SurnameAllDied,SurnameOneLived
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.0,0.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.0,0.0,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q,0.0,1.0,0.0
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0.0,0.0,0.0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0.0,0.0,1.0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0.0,0.0,0.0


## Ocenjevanje atributov

## Napovedovanje

### Random forest

In [162]:
x3 = x[["Minor", "SurnameAllDied", "female"]]
x3test = xtest[["Minor", "SurnameAllDied", "female"]]
clf = RandomForestClassifier()
clf.fit(x3,y)
r = clf.predict(x3test)

In [163]:
resoult = test[["PassengerId"]]
resoult["Survived"] = r
resoult.to_csv("Predictions/rfc3.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### Naive Bayes

In [77]:
gnb = GaussianNB()
gnb.fit(x,y)
tt = gnb.predict(xtest)

In [78]:
resoult2 = test[["PassengerId"]]
resoult2["Survived"] = tt
resoult2 == resoult
#resoult2.to_csv("Predictions/nbc2.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,PassengerId,Survived
0,True,True
1,True,False
2,True,True
3,True,False
4,True,False
...,...,...
413,True,True
414,True,True
415,True,True
416,True,True


## Napovedovanje + testiranje

### Random forest

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from itertools import combinations

In [139]:
maxS = 0.65
maxT = []
for s in range(3,12):
    for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch", "FamilySize", "female", "male"], s):
        x1 = x[np.array(a)]
        ca = 0
        for i in range(7):
            x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.33)
            clf = RandomForestClassifier()
            clf.fit(x_train, y_train)
            r = clf.predict(x_test)
            ca += accuracy_score(y_test, r)
        current = ca / 7
        if maxS < current:
            maxS = current
            maxT = a
        print(current)
    print(s)

385
0.7911622276029056
0.8123486682808717
0.8056900726392252
0.8159806295399515
0.8020581113801454
0.7808716707021793
0.8165859564164649
0.801452784503632
0.8202179176755447
0.8159806295399515
0.8099273607748183
0.8135593220338982
0.633777239709443
0.7723970944309927
0.7972154963680387
0.7651331719128329
0.7748184019370461
0.8099273607748184
0.7941888619854721
0.7693704600484262
0.7893462469733656
0.7966101694915254
0.8099273607748183
0.7978208232445522
0.789951573849879
0.7923728813559322
0.8008474576271185
0.7639225181598063
0.7723970944309928
0.7808716707021792
0.7832929782082324
0.7772397094430993
0.7730024213075061
5
0.6682808716707022
0.6688861985472155
0.6719128329297821
0.6864406779661015
0.7984261501210652
0.8081113801452784
0.6815980629539952
0.6900726392251816
0.675544794188862
0.8044794188861984
0.8020581113801452
0.6840193704600485
0.6682808716707022
0.8056900726392252
0.7832929782082324
0.6912832929782082
0.8002421307506052
0.7984261501210653
0.7887409200968524
0.79358353

In [140]:
maxS

0.8401937046004841

In [141]:
maxT

('Minor', 'SurnameAllDied', 'female')

### Naive bayes

In [133]:
x

,Pclass,Age,Fare,Minor,SurnameAllDied,SurnameOneLived,SibSp,Parch,FamilySize,female,male
0,3,22.0,7.2500,0.0,0.0,0.0,1,0,1,0.0,1.0
1,1,38.0,71.2833,0.0,0.0,0.0,1,0,1,1.0,0.0
2,3,26.0,7.9250,0.0,0.0,0.0,0,0,0,1.0,0.0
3,1,35.0,53.1000,0.0,0.0,0.0,1,0,1,1.0,0.0
4,3,35.0,8.0500,0.0,0.0,0.0,0,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
885,3,39.0,29.1250,0.0,1.0,0.0,0,5,5,1.0,0.0
886,2,27.0,13.0000,0.0,0.0,0.0,0,0,0,0.0,1.0
887,1,19.0,30.0000,0.0,0.0,1.0,0,0,0,1.0,0.0
889,1,26.0,30.0000,0.0,0.0,0.0,0,0,0,0.0,1.0


In [160]:
ca = 0
x3 = x[["Minor", "SurnameAllDied", "female"]]
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x3, y, test_size=0.33)
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    r = gnb.predict(x_test)
    ca += accuracy_score(y_test, r)
print(ca / 10)

0.84364406779661


In [122]:
for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"], 2):
    train[np.array(a)]

In [125]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Minor',
       'SurnameAllDied', 'SurnameOneLived'],
      dtype='object')

In [98]:
for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"], 2):
    print(a)

('Pclass', 'Age')
('Pclass', 'Fare')
('Pclass', 'Minor')
('Pclass', 'SurnameAllDied')
('Pclass', 'SurnameOneLived')
('Pclass', 'SibSp')
('Pclass', 'Parch')
('Age', 'Fare')
('Age', 'Minor')
('Age', 'SurnameAllDied')
('Age', 'SurnameOneLived')
('Age', 'SibSp')
('Age', 'Parch')
('Fare', 'Minor')
('Fare', 'SurnameAllDied')
('Fare', 'SurnameOneLived')
('Fare', 'SibSp')
('Fare', 'Parch')
('Minor', 'SurnameAllDied')
('Minor', 'SurnameOneLived')
('Minor', 'SibSp')
('Minor', 'Parch')
('SurnameAllDied', 'SurnameOneLived')
('SurnameAllDied', 'SibSp')
('SurnameAllDied', 'Parch')
('SurnameOneLived', 'SibSp')
('SurnameOneLived', 'Parch')
('SibSp', 'Parch')
